# Multiplos clientes
***

Basicamente faremos varias conexões ao servidor com multiplos clientes para ver o erro que ocorre e como trata-los através de servidores baseados em threads.

***
### Lado Cliente
***

In [1]:
from socket import *
from threading import Thread

***

In [2]:
class Client(Thread):
    """Classe que gera os clientes."""
    
    def __init__(self, client_id, server, port, *messages):
        # Número de identificação do cliente
        self.client_id = client_id
        
        # Servidor a ser conectado
        self.server = server
        
        # Porta para ser usada
        self.port = port
        
        # Mensagens a serem colocadas
        self.messages = messages
        
        Thread.__init__(self)
        
    def run(self):
        # Criamos o socket e o conectamos ao servidor
        connection = socket(AF_INET, SOCK_STREAM)
        connection.connect((self.server, self.port))
        
        # Mandamos a mensagem linha por linha
        for line in self.messages:
            connection.send(line)
            
            # Depois de mandar uma linha esperamos a resposta do servidor
            data = connection.recv(1024)
            print('Cliente {fulano} recebeu: {data}'.format(fulano=self.client_id, data=data))
            
        connection.close()

***

In [3]:
# Configurações de conexão do servidor
# O nome do servidor pode ser o endereço de
# IP ou o domínio (www.algo.com)
server_host = 'localhost'
server_port = 5000

***

In [4]:
# Mensagem a ser mandada codificada em bytes
message = [b'Ola mundo da internet!']

***

In [5]:
# Criamos vários clientes conectados ao mesmo servidor
def connection():
    for client in range(20):
        Client(client, server_host, server_port, *message).start()
        
    print("Geramos todos os clientes!")

***

In [6]:
# Algumas threads não consegue se conectar ao servidor devido
# ao excesso de processamento no servidor
# Para resolver os servidores devem ser baseados em threads, forks e etc...
connection()

Geramos todos os clientes!
Cliente 0 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 1 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 2 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 3 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 4 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 5 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 6 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 17 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 14 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 15 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 8 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 10 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 12 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 19 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 9 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 16 recebeu: b'Resposta => Ola mundo da internet!'
Cliente 7 recebeu: b'Resposta => Ola mundo da internet

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-c8d17607a042>", line 29, in run
    data = connection.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-c8d17607a042>", line 29, in run
    data = connection.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer

Exception in thread Thread-22:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-2-c8d17607a042>", line 29, in run
    data = connection.recv(1024)
ConnectionResetError: [Errno 104] Connection reset by peer



***

In [7]:
# Conexão através de um servidor baseado em threads
connection()

Geramos todos os clientes!
Cliente 0 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"
Cliente 3 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"Cliente 1 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"Cliente 2 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"


Cliente 4 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"Cliente 7 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"Cliente 5 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"


Cliente 8 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"
Cliente 6 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"
Cliente 9 recebeu: b"Resposta eco => b'Ola mundo da internet!' as Sat Jul 22 18:43:57 2017"
Cliente 10 recebeu: b"Resposta eco => b'Ola mundo da 

***
### servidor baseado em threads
***

O lado do servidor abre um TCP/IP em uma porta, espera por uma mensagem de um cliente, e manda essa mensagem de volta como resposta. Usamos aqui a biblioteca **socketserver** para realizar este trabalho. Esta biblioteca fornece TCPServer, THreadingTCPServer e ForkingTCPServer, UDP e variações destes, entre outras coisas, e redireciona cada cliente para um **request handler** para utilizar se do método **handle** para lidar com o requisito do cliente.

In [8]:
# Importa os modulos
from socketserver import BaseRequestHandler, ThreadingTCPServer
import time

***

In [9]:
# Configura o servidor com o host e a porta
def configure_server():
    server_host = 'localhost'
    server_port = 5000
    return (server_host, server_port)

***

In [10]:
# Devolve a hora atual
def now():
    return time.ctime(time.time())

***

In [11]:
# Classe que lida com as requisições do cliente
class HandlesClientRequests(BaseRequestHandler):
    
    def handle(self):
        # Lida com cada requisição do cliente
        print(self.client_address, now())
        
        # Simula processamento dos dados
        time.sleep(5)
    
        while True:
            # Recebe dados enviados pelo cliente
            data = self.request.recv(1024)
        
            # Se não receber nada paramos o loop
            if not data: break
            
            # Escreve a resposta
            answer = 'Resposta eco => {data} as {time}'.format(data=data, time=now())
        
            # Servidor manda de volta a resposta
            self.request.send(answer.encode())
        
        # Fecha a conexão criada depois de responder o cliente
        self.request.close()

***

In [13]:
# Cria uma thread server e lida com a entrada e requisitos do cliente
def main():
    ip_address = configure_server()
    server = ThreadingTCPServer(ip_address, HandlesClientRequests)
    server.serve_forever()

***
### Selecionar tarefas com o select
***


Threads não rodam realmente em paralelo a não ser que você tenha vários CPUs, o que realmente acontece é que o sistema operacional escolhe entre as tarefas a serem executadas aquela a ser executada, e vai trocando entre as threads. O seu sistema operacional faz isso tão rápido que você tem a ilusão de que as threads estão sendo executadas em paralelo. Esse processo é chamado de multiplexing

Porém podemos fazer o python selecionar as tarefas a serem executadas até todas serem completadas. Servidores podem aplicar essa técnica para lidar com múltiplos clientes ao mesmo tempo sem utilizar threads ou forks

Para fazer isso usamos o módulo select da biblioteca padrão, vamos realizar o multiplexing sem uso de nada do sistema operacional.

Esse servidor irá lidar com múltiplos clientes em paralelo com o select. Usar select para manualmente lidar com um conjunto de sockets: Socket principais que aceitam novas conexões, e sockets de input conectadas para aceitar clientes.

In [14]:
# Importar o select e o socket
from select import select
from socket import socket, AF_INET, SOCK_STREAM
import time

***

In [15]:
def now(): return time.ctime(time.time())

***

In [16]:
# Configurações do servidor
server_host = 'localhost'
server_port = 5002

***

In [17]:
# Número de sockets usados
socket_number = 2

***

In [18]:
# Lista de sockets criados por função de cada socket
main_socket = []
read_socket = []
write_socket = []

***

In [19]:
# Cria um socket pada cada função
for i in range(socket_number):
    # Configura um socket TCP/IP
    connection_socket = socket(AF_INET, SOCK_STREAM)
    
    # Configura o socket
    connection_socket.bind((server_host, server_port))
    connection_socket.listen(5)
    
    # Adiciona a lista de sockets principais e leitores
    main_socket.append(connection_socket)
    read_socket.append(connection_socket)
    
    # Aumenta o valor da porta para mudar o próximo socket
    server_port += 1

***

In [20]:
def main():
    print("Loop de seleção de socket iniciado!")
    
    while True:
        # Vemos todos os sockets legíveis e escrevíveis e os selecionamos
        readable_socket, writeable_socket, exceptions = select(read_socket, write_socket, [])
        
        # Para cada socket legível
        for socket_object in readable_socket:
            # Se ele é um socket principal
            if socket_object in main_socket:
                # Aceita o socket
                connection_socket, ip_address = socket_object.accept()
                # Imprime as conexões
                print("Conecta:", ip_address, id(connection_socket))
                # E o coloca no socket de leitura
                read_socket.append(connection_socket)
            else:
                # Lemos o que está no socket
                data = socket_object.recv(1024)
                
                # Imprime a mensagem recebida
                print("\tRecebeu", data, "em", id(socket_object))
                
                # Se não recebermos nada
                if not data:
                    # Fechamos o socket
                    socket_object.close()
                    # E o removemos do socket de leitura
                    read_socket.remove(socket_object)
                else:
                    # Preparamos uma resposta para ser enviado
                    answer = 'Resposta eco => %s as %s' % (data, now())
                    socket_object.send(answer.encode())
                    